In [1]:
import pandas as pd
import numpy as np
import pandas_gbq
import os.path
from pandas.io.json import json_normalize
from datetime import datetime

In [2]:
query = """

WITH
  max_date AS (
  SELECT
    merchant_id,
    MAX(date) MaxDate,
  FROM
    bharatpe-analytics-prod.bharatpe_mongo_data.merchant_psp_dump
  WHERE
    date>'2018-01-01'
  GROUP BY
    merchant_id )
SELECT
  app.merchant_id,
  app_details
FROM
  bharatpe-analytics-prod.bharatpe_mongo_data.merchant_psp_dump app
JOIN
  max_date
ON
  max_date.merchant_id = app.merchant_id
  AND max_date.MaxDate = app.date
WHERE
  date>'2018-01-01'
  AND app.merchant_id IN (
  SELECT
    merchant_id
  FROM (
    SELECT
      merchant_id,
      COUNT(DISTINCT id) AS txns,
      SUM(amount) AS tpv
    FROM
      bharatpe-analytics-prod.payin.transactions
    WHERE
      status = "SUCCESS"
      AND DATE(payment_timestamp) >= DATE_SUB(CURRENT_DATE(), INTERVAL 90 DAY)
    GROUP BY
      1
    HAVING
      txns >=5
      AND tpv >= 150 ) )

"""

In [3]:
df = pandas_gbq.read_gbq(query)

Downloading: 100%|██████████| 1842735/1842735 [06:29<00:00, 4727.68rows/s]


In [4]:
# df['partition'] = df['merchant_id'].mod(100)

In [5]:
app_sessions = """

WITH
      merchant_list AS (
      SELECT
        pt.merchant_id,
        dm.mid,
        COUNT(DISTINCT id) AS txns,
        SUM(amount) AS tpv
      FROM
        bharatpe-analytics-prod.payin.transactions pt
      JOIN
        `bharatpe-analytics-prod.bharatpe_data_platfrom.universal_merchant_data_mart` dm
      ON
        pt.merchant_id = dm.merchant_id
      WHERE
        status = "SUCCESS"
        AND dm.merchant_created_at >= "2018-01-01"
        AND DATE(payment_timestamp) >= DATE_SUB(CURRENT_DATE(), INTERVAL 90 DAY)
      GROUP BY
        1,
        2
      HAVING
        txns >=5 and tpv >= 150),
      app1 AS (
      SELECT
        mid,
        DATE(created_at) AS created_at,
        count(session_id) as sessions,
        CURRENT_DATE() AS today
       
      FROM
        bharatpe-analytics-prod.bharatpe_analytics_data.app_event
      WHERE
        DATE(created_at) >= DATE_SUB(CURRENT_DATE(), INTERVAL 31 DAY)
      GROUP BY
        1,
        2 ),
      app2 AS (
      SELECT
        mid,
        SUM(CASE
            WHEN created_at < today AND created_at >= DATE_SUB(today, INTERVAL 7 DAY) THEN sessions
          ELSE
          0
        END
          ) AS w1_sessions,
        SUM(CASE
            WHEN created_at < today AND created_at >= DATE_SUB(today, INTERVAL 30 DAY) THEN sessions
          ELSE
          0
        END
          ) AS m1_sessions
      FROM
        app1
      GROUP BY
        1 )
    SELECT
      ml.merchant_id,
      w1_sessions,
      m1_sessions
    FROM
      app2
    JOIN
      merchant_list AS ml
    ON
      app2.mid = ml.mid
"""

In [6]:
app_data = pandas_gbq.read_gbq(app_sessions)

Downloading: 100%|██████████| 2462243/2462243 [00:48<00:00, 50985.79rows/s]


In [7]:
session_dic = app_data.set_index("merchant_id")['w1_sessions'].to_dict()

In [8]:
del (app_data)

In [9]:
def apps(x): 
    try:
        ev = eval(x[1:-1])
        result = [ev[i]['appName'] for i in range(0,len(ev))]
    except:
        result = []
    return result

def packages(x): 
    try:
        ev = eval(x[1:-1])
        result = [ev[i]['packageName'] for i in range(0,len(ev))]
    except:
        result = []
    return result
    
    # return [eval(x[1:-1])[i]['appName'] for i in range(0,len(eval(x[1:-1])))]
    

In [10]:
start = datetime.now()

In [11]:
df['app_names'] = df['app_details'].apply(lambda x :apps(x))

In [12]:
# df['package_names'] = df['app_details'].apply(lambda x :packages(x))

In [13]:
end = datetime.now()

In [14]:
end - start

datetime.timedelta(seconds=363, microseconds=138304)

In [15]:
df.to_csv("apps.csv", index = False)

In [16]:
# df = pd.read_csv("apps.csv")

In [17]:
d = {}
for item in df['app_names']:
    for el in item:
        if el in d:
            d[el] +=1
        else:
            d[el]=1      

In [18]:
relevant_apps = [
 'PhonePe',
 'Paytm',
 'GPay',
 'PhonePe Business',
 'Paytm for Business',
 'Google Pay for Business',
 'WhatsApp\xa0Business',
 'Bajaj Finserv',
 'Duo',
 'Khatabook',
 'Kotak Bank',
 'HDFC Bank',
 'CRED',
 'Axis Mobile',
 'MobiKwik',
 'iMobile Pay',
 'OkCredit',
 'BHIM',
 'Amazon Pay for Business',
 'LinkedIn',
 ]

In [19]:
sorted(d.items(), key=lambda x: -x[1])[:5]
# [i[0] for i in  sorted(d.items(), key=lambda x: -x[1])[:100]]

[('BharatPe', 1841874),
 ('WhatsApp', 1702639),
 ('PhonePe', 1502206),
 ('Truecaller', 1230518),
 ('Drive', 1194357)]

In [20]:
apps_list = ["com.loan.cash.credit.easy.dhan.quick.udhaar.lend.game.jaldi.paisa.borrow.rupee.play.kredit","com.kreditbee.android",\
             "com.portal.hcin","com.capitalfirst","com.fastbanking","com.whizdm.moneyview.loans","com.indigo.hdfcloans",\
             "com.nucleus.finnone.mobile.mserve.hdb.eng","com.kreditzy.android","in.zestmoney.android.zestlife","com.fincare.mb"\
             ,"com.muthootfinance.imuthoot","com.whizdm.moneyview","com.tvscs.tvscreditapp","com.mycash.moneytap.app","com.manappuram.b2c"\
             ,"com.indialends.android","com.nanocred.cash.kredit.loan.dhan.fast.quick.easy.borrow.lend.paisa","in.loanfront.android",\
             "com.jc.icredit","com.naviapp","com.okcash.liufang","com.kn.instarupee","com.xu.pineapple.free","com.gocash.liufang",\
             "com.nexgen.nexmoney","com.lucifer.onepay","com.creditmantri","videos.share.rozdhan","com.snapwork.tcl","com.indiabulls.dhanibiz"\
             ,"com.lendingadda.jienavi","in.cashmama.app","com.GotoCash","com.earlysalary.android","com.frogx.ieasyloan",\
             "com.muthootfincorp.mymuthoot","com.iifl.mobile.hfc","com.rong360.android_abroad_loans","com.mycytwo.zy","com.lending.rupeehub",\
             "com.getrupee.indiaapp","com.wisekredit.wisekredit","com.LendingKart","com.simple.cm","com.flexiloan","com.particle.videoplayapp",\
             "com.sbi.apps.sbi_loans","com.balancehero.truebalance"]

In [21]:
df['relevant_apps'] = df['app_names'].apply(lambda x:len(set(x).intersection(relevant_apps)))

In [22]:
df['total_apps'] = df['app_names'].apply(lambda x:len(x))

In [23]:
df['w1_sessions'] = df['merchant_id'].map(session_dic)

In [24]:
df['w1_sessions_bins'] = pd.cut(df['w1_sessions'], bins=[df['w1_sessions'].describe()['min'], df['w1_sessions'].describe()['25%']\
                                                       , df['w1_sessions'].describe()['50%'], df['w1_sessions'].describe()['75%']\
                                                       ,df['w1_sessions'].describe()['max']])

In [25]:
df['total_apps_bins'] = pd.cut(df['total_apps'], bins=[df['total_apps'].describe()['min'], df['total_apps'].describe()['25%']\
                                                       , df['total_apps'].describe()['50%'], df['total_apps'].describe()['75%']\
                                                       ,df['total_apps'].describe()['max']])

In [26]:
df['relevant_apps_bins'] = pd.cut(df['relevant_apps'], bins=[df['relevant_apps'].describe()['min'], df['relevant_apps'].describe()['25%']\
                                                       , df['relevant_apps'].describe()['50%'], df['relevant_apps'].describe()['75%']\
                                                       ,df['relevant_apps'].describe()['max']])

In [27]:
# pd.pivot_table(df, values ='merchant_id', index =['total_apps_bins'],columns =['relevant_apps_bins'], aggfunc = pd.Series.nunique)

In [28]:
pd.pivot_table(df, values ='merchant_id', index =['w1_sessions_bins'],columns =['relevant_apps_bins'], aggfunc = pd.Series.nunique)

relevant_apps_bins,"(0.0, 2.0]","(2.0, 4.0]","(4.0, 6.0]","(6.0, 18.0]"
w1_sessions_bins,,,,
"(0.0, 30.0]",30683,43438,31528,22659
"(30.0, 172.0]",101587,139604,100042,79086
"(172.0, 480.0]",98131,137263,102566,83838
"(480.0, 345915.0]",85454,133036,110935,96484


In [29]:
# c1 = df['relevant_apps'] >= 4
# c2 = df['total_apps']>= 38
# c = (c1) & (c2)
# df['Channel'] = np.where(c, 'Digital','Offline')

c1 = df['relevant_apps'] >= 4
c2 = df['w1_sessions']>= 480
c = (c1) & (c2)
df['Channel'] = np.where(c, 'Digital','Offline')

In [30]:
df[df['Channel'] == 'Digital'][['merchant_id']].to_csv("digital_merchants.csv", index = False)